# julia itteration solver sparse src

In [46]:
import LinearAlgebra: mul!, ldiv!
import Base: getindex, iterate
using SparseArrays

struct DiagonalIndices{Tv, Ti <: Integer}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::Vector{Ti}

    function DiagonalIndices{Tv,Ti}(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti}
        # Check square?
        diag = Vector{Ti}(undef, A.n)

        for col = 1 : A.n
            r1 = Int(A.colptr[col])
            r2 = Int(A.colptr[col + 1] - 1)
            r1 = searchsortedfirst(A.rowval, col, r1, r2, Base.Order.Forward)
            if r1 > r2 || A.rowval[r1] != col || iszero(A.nzval[r1])
                throw(LinearAlgebra.SingularException(col))
            end
            diag[col] = r1
        end 

        new(A, diag) #
    end
end

DiagonalIndices(A::SparseMatrixCSC{Tv,Ti}) where {Tv,Ti} = DiagonalIndices{Tv,Ti}(A)
@inline getindex(d::DiagonalIndices, i::Int) = d.diag[i]


struct FastLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct FastUpperTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct StrictlyUpperTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct StrictlyLowerTriangular{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end

struct OffDiagonal{Tv,Ti}
    matrix::SparseMatrixCSC{Tv,Ti}
    diag::DiagonalIndices{Tv,Ti}
end


function forward_sub!(F::FastLowerTriangular, x::AbstractVector)
    A = F.matrix
    @inbounds for col = 1 : A.n
        idx = F.diag[col]
        x[col] /= A.nzval[idx] # ok
        for i = idx + 1 : (A.colptr[col + 1] - 1) #colptr인데 lower triangular이기 때문에 해당 col의 diagonal 아래 개수가나옴.
            x[A.rowval[i]] -= A.nzval[i] * x[col] # 이 term으로 x[n] 계산할때 그이전텀들이 다 마이너스 되어서 있음. 
        end
    end
    x
end

function backward_sub!(F::FastUpperTriangular, x::AbstractVector)
    A = F.matrix

    @inbounds for col = A.n : -1 : 1

        # Solve for diagonal element
        idx = F.diag[col]
        x[col] = x[col] / A.nzval[idx]

        # Substitute next values involving x[col]
        for i = A.colptr[col] : idx - 1
            x[A.rowval[i]] -= A.nzval[i] * x[col]
        end
    end

    x
end




backward_sub! (generic function with 1 method)

# Custom src

In [44]:

#(α×A×X)+(β×y)  
function f_mul(α::T, O::DiagonalIndices, x::AbstractVector, β::T, b::AbstractVector ) where {T}
    # Specialize for β = 0 and β = 1
    A = O.matrix
    y = copy(b)

    if β != one(T)
        if iszero(β)
            fill!(y, zero(T))
        else
            lmul!(β, y) 
        end
    end

    @inbounds for col = 1 : A.n
        αx = α * x[col]
        diag_index = O.diag[col]
        for j = A.colptr[col] : A.colptr[col + 1] - 1
            y[A.rowval[j]] += A.nzval[j] * αx
        end
    end

    y
end




function m_sor!(A, D::DiagonalIndices, w)
    for d_idx in D.diag 
        A.nzval[d_idx]  *= (1/w)
    end
    @inbounds for col = 1 : A.n
        for j = A.colptr[col] :  A.colptr[col + 1] - 1
            if A.rowval[j] < col 
                A.nzval[j] = 0
            end
        end
    end
    
end


function itter_sor!(F::FastLowerTriangular, D::DiagonalIndices,
                        x::AbstractVector, b::AbstractVector, max_itter)
    A = D.matrix
    T = eltype(b)
    r =zeros(A.n)
    
    r[:] = b - A * x
    
    for i = 1 : max_itter 
        if norm( r, 2) < 10^-8 
            return x
        end
        x[:] += forward_sub!(F, r)
        #r[:] = b - A * x
        r[:] = f_mul(-one(T), D, x, one(T), b)
    end
    
    x
end


function k3_sor(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    m_sor!(m_sor, D, w)
    D_ms = DiagonalIndices(m_sor)
    itter_sor!(FastLowerTriangular(m_sor ,D_ms), D, x , b, maxiter)
end


function gm_sqrt_diag_mul!(D::DiagonalIndices, b::AbstractVector, w)
    A = D.matrix
    γ = sqrt((2/w) -1)
    for idx in D.diag 
        b[A.rowval[idx]] *=  (γ * sqrt(A.nzval[idx]))
    end
end

function itter_ssor!(F::FastLowerTriangular, U::FastUpperTriangular, D::DiagonalIndices,
                        D_t::DiagonalIndices, x::AbstractVector, b::AbstractVector
                        , w,  max_itter)
    
    A = D.matrix
    A_t = D_t.matrix
    
    T = eltype(b)
    r_1 = zeros(A.n)
    r_2 = zeros(A.n)
    
    
    gm_sqrt_diag_mul!(D,b,w)
    #
        
    for i = 1 : max_itter 
        r_1[:] = f_mul(-one(T), D, x, one(T), b)
        x[:] += forward_sub!(F, r_1)
        r_2[:] = f_mul(-one(T), D_t, x, one(T), b)
        x[:] += backward_sub!(U, r_2)
    end
    
    x
end

function itter_cheby_ssor!(F::FastLowerTriangular, U::FastUpperTriangular, D::DiagonalIndices,
                        D_t::DiagonalIndices, x::AbstractVector, b::AbstractVector
                        , w,  max_itter)
    
    A = D.matrix
    A_t = D_t.matrix
    
    T = eltype(b)
    r_1 = zeros(A.n)
    r_2 = zeros(A.n)
    
    
    gm_sqrt_diag_mul!(D,b,w)
    #
        
    for i = 1 : max_itter 
        r_1[:] = f_mul(-one(T), D, x, one(T), b)
        x[:] += forward_sub!(F, r_1)
        r_2[:] = f_mul(-one(T), D_t, x, one(T), b)
        x[:] += backward_sub!(U, r_2)
    end
    
    x
end



function k3_ssor(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    D_t = DiagonalIndices(sparse(A'))
    
    m_sor!(m_sor, D, w)
    
    D_ms = DiagonalIndices(m_sor)
    m_sor_t = sparse(m_sor')
    D_ms_t = DiagonalIndices(m_sor_t)
    
    itter_ssor!(FastLowerTriangular(m_sor ,D_ms), FastUpperTriangular(m_sor_t,D_ms_t),
                    D, D_t, x , b, w, maxiter)
end


function k3_cheby_ssor(A, b::AbstractVector, w, maxiter)
    x = zeros(A.n)
    m_sor = copy(A)
    D = DiagonalIndices(A)
    D_t = DiagonalIndices(sparse(A'))
    
    m_sor!(m_sor, D, w)
    
    D_ms = DiagonalIndices(m_sor)
    m_sor_t = sparse(m_sor')
    D_ms_t = DiagonalIndices(m_sor_t)
    
    itter_cheby_ssor!(FastLowerTriangular(m_sor ,D_ms), FastUpperTriangular(m_sor_t,D_ms_t),
                    D, D_t, x , b, w, maxiter)
end








k3_ssor (generic function with 1 method)

# Test code

In [40]:
using BenchmarkTools, IterativeSolvers, LinearAlgebra, MatrixDepot, Random

Random.seed!(280)

n = 100
# Poisson matrix of dimension n^2=10000, pd and sparse
A = matrixdepot("poisson", n)
@show typeof(A)
# dense matrix representation of A
#Afull = convert(Matrix, A)
@show typeof(Afull)
# sparsity level
count(!iszero, A) / length(A)
b = randn(n^2)

typeof(A) = SparseMatrixCSC{Float64,Int64}
typeof(Afull) = Array{Float64,2}

10000-element Array{Float64,1}:
  0.12623784496408763 
 -2.3468794813834966  
  1.9166090582536102  
 -0.23920888512829233 
 -0.5784508270929073  
 -1.1278278196026448  
  1.1478628422667982  
 -1.354705173870581   
 -0.23706547458394342 
 -0.6809935026697     
 -0.8826696457022515  
  1.7138424693341203  
 -0.7339523682572253  
  ⋮                   
 -1.1242756459799825  
 -1.5840165152654142  
 -0.18205667606452444 
 -0.7596266189004605  
 -1.4396141684820316  
  0.004180129114450185
 -0.07636458207051988 
 -1.9658459863745268  
 -1.3457051266647149  
  0.3120272656011855  
  0.6456518202208312  
 -0.28866613765800436 

In [37]:
@benchmark sor($A, $b, 0.75, maxiter=10000)

BenchmarkTools.Trial: 
  memory estimate:  23.19 MiB
  allocs estimate:  20010
  --------------
  minimum time:     173.797 s (0.00% GC)
  median time:      173.797 s (0.00% GC)
  mean time:        173.797 s (0.00% GC)
  maximum time:     173.797 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

In [33]:
norm(sor(A, b, 0.75, maxiter=10000) - k3_sor(A, b, 0.75, 10000) ,2)

1.0148607659440114e-13

In [38]:
@benchmark k3_sor($A, $b, 0.75, 10000)

BenchmarkTools.Trial: 
  memory estimate:  223.65 GiB
  allocs estimate:  60022
  --------------
  minimum time:     310.587 s (6.54% GC)
  median time:      310.587 s (6.54% GC)
  mean time:        310.587 s (6.54% GC)
  maximum time:     310.587 s (6.54% GC)
  --------------
  samples:          1
  evals/sample:     1

In [49]:
ssor(A, b, 0.75, maxiter=10000)

10000-element Array{Float64,1}:
 -0.9050094883584878 
 -3.4713912641633646 
  1.1984008009581797 
 -2.668923325717958  
 -4.193348307248316  
 -3.306490895033126  
  1.4809277319484275 
 -1.4172686448831753 
 -1.6909200279824679 
 -3.622925051912488  
 -3.9519896751431864 
 -0.09971979707153485
 -3.1545484324825592 
  ⋮                  
 -5.285415167008249  
 -8.14863426488505   
 -6.204506828781212  
 -7.916034216947419  
 -9.924735919251754  
 -8.256349629803367  
 -6.848395083248771  
 -8.06340662185411   
 -5.546557386866866  
 -0.6279507262969105 
  1.3084724363952853 
  0.7357327521837822 

In [47]:
k3_ssor(A, b, 0.75, 10000)

10000-element Array{Float64,1}:
 -0.9050094883584878 
 -3.4713912641633646 
  1.19840080095818   
 -2.6689233257179583 
 -4.193348307248317  
 -3.3064908950331255 
  1.4809277319484273 
 -1.4172686448831755 
 -1.690920027982468  
 -3.622925051912488  
 -3.9519896751431864 
 -0.09971979707153478
 -3.1545484324825597 
  ⋮                  
 -5.285415167008249  
 -8.14863426488505   
 -6.204506828781212  
 -7.916034216947418  
 -9.924735919251756  
 -8.256349629803369  
 -6.848395083248771  
 -8.06340662185411   
 -5.546557386866866  
 -0.6279507262969105 
  1.3084724363952853 
  0.735732752183782  

In [50]:
@benchmark k3_ssor($A, $b, 0.75, 10000)

BenchmarkTools.Trial: 
  memory estimate:  4.48 GiB
  allocs estimate:  120043
  --------------
  minimum time:     4.471 s (4.75% GC)
  median time:      4.526 s (4.93% GC)
  mean time:        4.526 s (4.93% GC)
  maximum time:     4.581 s (5.12% GC)
  --------------
  samples:          2
  evals/sample:     1

In [52]:
@benchmark ssor($A, $b, 0.75, maxiter=10000)

BenchmarkTools.Trial: 
  memory estimate:  859.84 KiB
  allocs estimate:  40012
  --------------
  minimum time:     2.838 s (0.00% GC)
  median time:      2.869 s (0.00% GC)
  mean time:        2.869 s (0.00% GC)
  maximum time:     2.901 s (0.00% GC)
  --------------
  samples:          2
  evals/sample:     1